# Exercise: Analyzing Chinook Database

Preparation I've done:
 - Retrieve the dataset and load it
 - Load the %sql extension and point it at the database
 - Display the tables and an example query

Additional steps you might take:
 - Add libraries for visualization (matplotlib, seaborn, plotly)
 - Add libraries for statistics (numpy)
 - Explore the dataset using SQL and/or pandas

----

1. Retrieve a list of all the tracks in the database, displaying only the track name and the name of the album it belongs to. Limit the result to the first 5 rows.
   > Operations: `SELECT`
2. Find the total number of customers from each country. Display the country name and the corresponding count. Order the results by the count in descending order.
   > Operations: `SELECT`, `COUNT`, `GROUP BY`, `ORDER BY`
3. Identify the top 5 genres with the highest number of tracks. Display the genre name along with the total number of tracks for each genre.
   > Operations: `SELECT`, `COUNT`, `GROUP BY`, `ORDER BY`
4. Determine the average invoice total for each customer, considering both the album and individual track purchases. Display the customer's first and last name along with the average invoice total. Order the results by the average invoice total in descending order.
   > Operations: `SELECT`, `AVG`, `JOIN`, `GROUP BY`, `ORDER BY`
5. Identify the customer who spent the most on music purchases. Display the customer's first and last name, along with the total amount spent.
   > Operations: `SELECT`, `SUM`, `JOIN`, `GROUP BY`, `ORDER BY`, `LIMIT`

In [2]:
#%pip install --upgrade pip

In [3]:
# Load chinook dataset and query it using SQL magic into pandas dataframes
%pip install pandas 
%pip install ipython-sql
#%pip install sqlite3
import pandas as pd
import sqlite3
%load_ext sql

# Load data
conn = sqlite3.connect("chinook.sqlite")

# Tell %sql about the database
%sql sqlite:///chinook.sqlite

# List tables in database
query = "SELECT name FROM sqlite_master WHERE type='table';"

# Read data into a Pandas DataFrame
tables = %sql $query

# Print head
display(tables)

# Query to get the first 5 rows of the `albums` table
result = %sql SELECT * FROM albums LIMIT 5;

# Display query result, note that Pandas DataFrame is returned!
display(result)


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


C:\Users\krbis\AppData\Local\Temp\ipykernel_23888\1596962900.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


 * sqlite:///chinook.sqlite
Done.


name
albums
sqlite_sequence
artists
customers
employees
genres
invoices
invoice_items
media_types
playlists


 * sqlite:///chinook.sqlite
Done.


AlbumId,Title,ArtistId
1,For Those About To Rock We Salute You,1
2,Balls to the Wall,2
3,Restless and Wild,2
4,Let There Be Rock,1
5,Big Ones,3


In [4]:
# Query to get the first 5 rows of the `tracks` table
TracksList = %sql SELECT name,albumid FROM tracks LIMIT 5

#join albums table to tracks table based on rows of tracks (left join), where albumid match in both tables... then select name(of track) and title(of album)
TracksList_Album = %sql SELECT name AS track_name, title AS album_title FROM tracks LEFT JOIN albums ON tracks.albumid = albums.albumid LIMIT 5

# Display query TracksList, note that Pandas DataFrame is returned!
display(TracksList)
display(TracksList_Album)



 * sqlite:///chinook.sqlite
Done.
 * sqlite:///chinook.sqlite
Done.


Name,AlbumId
For Those About To Rock (We Salute You),1
Balls to the Wall,2
Fast As a Shark,3
Restless and Wild,3
Princess of the Dawn,3


track_name,album_title
For Those About To Rock (We Salute You),For Those About To Rock We Salute You
Balls to the Wall,Balls to the Wall
Fast As a Shark,Restless and Wild
Restless and Wild,Restless and Wild
Princess of the Dawn,Restless and Wild


In [18]:
TotCustomers = %sql SELECT Country, COUNT(Country) AS CountryCount FROM customers GROUP BY Country ORDER BY CountryCount DESC
display(TotCustomers)


 * sqlite:///chinook.sqlite
Done.


Country,CountryCount
USA,13
Canada,8
France,5
Brazil,5
Germany,4
United Kingdom,3
Portugal,2
India,2
Czech Republic,2
Sweden,1


In [50]:
GenreList = %sql SELECT genres.Name, COUNT(genres.Name) AS GenreCount FROM genres LEFT JOIN tracks ON genres.genreid = tracks.genreid GROUP BY genres.Name ORDER BY GenreCount DESC LIMIT 5
display(GenreList)

 * sqlite:///chinook.sqlite
Done.


Name,GenreCount
Rock,1297
Latin,579
Metal,374
Alternative & Punk,332
Jazz,130


In [129]:
#use billing address to join the customers table with the Invoices table, pull First and Last Name, and gorup by Name. Aggregate based on Group by (joined by Name). Order by descending to get highest invoice to lowest rounded to 2 decimal points for monetary significance
AvgInvoice = %sql select customers.FirstName, customers.LastName, ROUND(AVG(Invoices.Total),2) AS AVGInvoiceTotal FROM Invoices LEFT JOIN customers on Invoices.BillingAddress = customers.Address GROUP BY customers.FirstName, customers.LastName ORDER BY AVGInvoiceTotal DESC
display(AvgInvoice)

 * sqlite:///chinook.sqlite
Done.


FirstName,LastName,AVGInvoiceTotal
Helena,Holý,7.09
Richard,Cunningham,6.8
Luis,Rojas,6.66
Hugh,O'Reilly,6.52
Ladislav,Kovács,6.52
Frank,Ralston,6.23
Fynn,Zimmermann,6.23
Julia,Barnett,6.23
Puja,Srivastava,6.11
Astrid,Gruber,6.09


In [132]:
#Similar to above, except rather than aggegating by averaging the invoices of each customer to determine average, I sum it up to see total purchase. Limit to 1 to see the highest spending customer as it is set in descending order
CustomerSpend = %sql select customers.FirstName, customers.LastName, SUM(Invoices.Total) AS SumInvoiceTotal FROM Invoices LEFT JOIN customers on Invoices.BillingAddress = customers.Address GROUP BY customers.FirstName, customers.LastName ORDER BY SumInvoiceTotal DESC LIMIT 1
display(CustomerSpend)

 * sqlite:///chinook.sqlite
Done.


FirstName,LastName,SumInvoiceTotal
Helena,Holý,49.62
